In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import torch

In [5]:
import torch
x = torch.rand(5, 3, 2)
print(x)

tensor([[[0.7038, 0.2525],
         [0.1881, 0.1776],
         [0.3897, 0.2838]],

        [[0.3392, 0.7200],
         [0.3295, 0.2739],
         [0.8637, 0.3751]],

        [[0.3129, 0.9907],
         [0.1120, 0.7530],
         [0.1531, 0.6820]],

        [[0.8642, 0.4108],
         [0.3396, 0.5499],
         [0.2431, 0.6501]],

        [[0.3468, 0.4088],
         [0.8634, 0.5926],
         [0.4690, 0.9144]]])


In [16]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [17]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [22]:
training_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [23]:
from torch import nn
from torch.utils.data import DataLoader

In [24]:
train_dataloader = DataLoader(training_data)

In [31]:
for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([1, 1, 28, 28])
Shape of y: torch.Size([1]) torch.int64


In [32]:
# To create model, must be done manually in a class
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 80),
            nn.ReLU(),
            nn.Linear(80, 80),
            nn.ReLU(),
            nn.Linear(80, 40),
            nn.ReLU(),
            nn.Linear(40, 20),
            nn.ReLU(),
            nn.Linear(20, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to('cpu')
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=80, bias=True)
    (1): ReLU()
    (2): Linear(in_features=80, out_features=80, bias=True)
    (3): ReLU()
    (4): Linear(in_features=80, out_features=40, bias=True)
    (5): ReLU()
    (6): Linear(in_features=40, out_features=20, bias=True)
    (7): ReLU()
    (8): Linear(in_features=20, out_features=10, bias=True)
  )
)


In [33]:
# Define loss function and optimizer (recall, model seeks to minimize loss using optimizer)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [34]:
# Create function that performs the fitting of model
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to('cpu'), y.to('cpu')

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [35]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to('cpu'), y.to('cpu')
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [36]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.235820  [    0/60000]
loss: 2.243317  [  100/60000]
loss: 2.467592  [  200/60000]
loss: 2.227604  [  300/60000]
loss: 2.447607  [  400/60000]
loss: 2.298117  [  500/60000]
loss: 2.223757  [  600/60000]
loss: 2.274035  [  700/60000]
loss: 2.221588  [  800/60000]
loss: 2.228010  [  900/60000]
loss: 2.550555  [ 1000/60000]
loss: 2.213685  [ 1100/60000]
loss: 2.413570  [ 1200/60000]
loss: 2.153628  [ 1300/60000]
loss: 2.151630  [ 1400/60000]
loss: 2.238556  [ 1500/60000]
loss: 2.545982  [ 1600/60000]
loss: 2.406492  [ 1700/60000]
loss: 2.430584  [ 1800/60000]
loss: 2.425204  [ 1900/60000]
loss: 2.271270  [ 2000/60000]
loss: 2.524377  [ 2100/60000]
loss: 2.298870  [ 2200/60000]
loss: 2.206094  [ 2300/60000]
loss: 2.272128  [ 2400/60000]
loss: 2.520601  [ 2500/60000]
loss: 2.255854  [ 2600/60000]
loss: 2.413193  [ 2700/60000]
loss: 2.256066  [ 2800/60000]
loss: 2.288887  [ 2900/60000]
loss: 2.268605  [ 3000/60000]
loss: 2.245214  [ 3100/60000]


loss: 1.623784  [27200/60000]
loss: 0.214937  [27300/60000]
loss: 0.952616  [27400/60000]
loss: 0.743447  [27500/60000]
loss: 0.645838  [27600/60000]
loss: 2.412195  [27700/60000]
loss: 0.867474  [27800/60000]
loss: 0.606958  [27900/60000]
loss: 0.134277  [28000/60000]
loss: 0.106250  [28100/60000]
loss: 1.203400  [28200/60000]
loss: 0.285153  [28300/60000]
loss: 0.342796  [28400/60000]
loss: 0.989748  [28500/60000]
loss: 0.487895  [28600/60000]
loss: 0.478303  [28700/60000]
loss: 0.425999  [28800/60000]
loss: 0.734009  [28900/60000]
loss: 0.544491  [29000/60000]
loss: 1.688296  [29100/60000]
loss: 0.537522  [29200/60000]
loss: 2.000805  [29300/60000]
loss: 1.755341  [29400/60000]
loss: 0.442056  [29500/60000]
loss: 0.639791  [29600/60000]
loss: 0.986411  [29700/60000]
loss: 2.109319  [29800/60000]
loss: 1.667615  [29900/60000]
loss: 0.571541  [30000/60000]
loss: 0.545271  [30100/60000]
loss: 0.181887  [30200/60000]
loss: 0.312464  [30300/60000]
loss: 1.035719  [30400/60000]
loss: 0.86

loss: 0.005855  [54800/60000]
loss: 0.051005  [54900/60000]
loss: 0.023456  [55000/60000]
loss: 0.043060  [55100/60000]
loss: 0.071852  [55200/60000]
loss: 0.004640  [55300/60000]
loss: 0.160916  [55400/60000]
loss: 0.158157  [55500/60000]
loss: 0.060879  [55600/60000]
loss: 0.008059  [55700/60000]
loss: 0.139655  [55800/60000]
loss: 0.033397  [55900/60000]
loss: 0.025600  [56000/60000]
loss: 0.011606  [56100/60000]
loss: 0.033751  [56200/60000]
loss: 3.349881  [56300/60000]
loss: 0.025990  [56400/60000]
loss: 0.027315  [56500/60000]
loss: 3.878091  [56600/60000]
loss: 0.030100  [56700/60000]
loss: 3.537750  [56800/60000]
loss: 0.038979  [56900/60000]
loss: 0.000256  [57000/60000]
loss: 0.567515  [57100/60000]
loss: 0.038640  [57200/60000]
loss: 0.107266  [57300/60000]
loss: 0.088119  [57400/60000]
loss: 0.001174  [57500/60000]
loss: 0.002655  [57600/60000]
loss: 0.045239  [57700/60000]
loss: 0.046911  [57800/60000]
loss: 0.316198  [57900/60000]
loss: 0.119014  [58000/60000]
loss: 0.81

NameError: name 'test_dataloader' is not defined

##### Yikes!  Ok now see why batch size is needed for PyTorch

In [37]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [38]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.376924  [    0/60000]
loss: 0.319363  [ 6400/60000]
loss: 0.241200  [12800/60000]
loss: 0.373960  [19200/60000]
loss: 0.368900  [25600/60000]
loss: 0.373416  [32000/60000]
loss: 0.215285  [38400/60000]
loss: 0.332208  [44800/60000]
loss: 0.297689  [51200/60000]
loss: 0.377219  [57600/60000]
Test Error: 
 Accuracy: 90.5%, Avg loss: 0.319167 

Epoch 2
-------------------------------
loss: 0.334014  [    0/60000]
loss: 0.307796  [ 6400/60000]
loss: 0.231161  [12800/60000]
loss: 0.371427  [19200/60000]
loss: 0.360897  [25600/60000]
loss: 0.348333  [32000/60000]
loss: 0.201599  [38400/60000]
loss: 0.341924  [44800/60000]
loss: 0.281649  [51200/60000]
loss: 0.369607  [57600/60000]
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.310508 

Epoch 3
-------------------------------
loss: 0.317849  [    0/60000]
loss: 0.300507  [ 6400/60000]
loss: 0.227408  [12800/60000]
loss: 0.369562  [19200/60000]
loss: 0.354561  [25600/60000]
loss: 0.335497  [32000/600